# Project 1: Pre-Code Hollywood Analysis
## Part 1: Loading and Filtering Core Movie Data (1929-1934)

This notebook begins our exploration of Pre-Code Hollywood cinema. Based on the historical context, we are defining the Pre-Code era as films released between **1929** and **1934**, inclusive.

**Objective:**
The goal of this first step is to load the primary dataset containing movie information (`title.basics.tsv`) and filter it down to *only* the movies released in our target era.

**Methodology:**
1.  **Load Data:** We will use the `pandas` library to load the `title.basics.tsv` file into a DataFrame. We must specify that the file is tab-separated (`sep='\t'`) and that the null value indicator is `\N`, as per the IMDb dataset documentation.
2.  **Filter by Type:** The dataset contains all types of titles (e.g., `tvEpisode`, `short`, `videoGame`). We will filter to keep only `titleType == 'movie'`.
3.  **Filter by Year:** We will then apply a date range filter to keep only movies where `startYear` is between 1929 and 1934.
4.  **Verify:** We will inspect the resulting DataFrame's shape, data types (`.info()`), and first few rows (`.head()`) to confirm our filtering was successful.

In [1]:
import pandas as pd
import os

print(f"Pandas version: {pd.__version__}")

# --- 1. Define File Paths ---
# Construct a relative path to our data directory.
# This makes the notebook portable, as it doesn't rely on absolute paths.
DATA_DIR = '../data/raw_imdb'
TITLES_FILE_PATH = os.path.join(DATA_DIR, 'title.basics.tsv')

print(f"Loading data from: {TITLES_FILE_PATH}")

# --- 2. Load the Core Titles Dataset ---
# We use pd.read_csv with a tab separator.
# `na_values='\\N'` correctly interprets IMDb's null value representation.
# `low_memory=False` is used here to prevent a DtypeWarning on loading,
# as pandas might initially see mixed types in some columns of this very large file.
titles_df = pd.read_csv(
    TITLES_FILE_PATH, 
    sep='\t', 
    na_values='\\N',
    low_memory=False
)

print("Successfully loaded title.basics.tsv.")
print("-" * 30)


# --- 3. Apply Filters for Pre-Code Movies ---
print("Filtering for Pre-Code era movies (1929-1934)...")

# Filter 1: Keep only rows where titleType is 'movie'.
is_movie = titles_df['titleType'] == 'movie'

# Filter 2: Keep only rows where startYear is between 1929 and 1934.
# .between() is an efficient and readable way to do this.
is_pre_code_era = titles_df['startYear'].between(1929, 1934, inclusive='both')

# Combine the filters and create a new DataFrame.
# Using .copy() is important here to avoid a SettingWithCopyWarning from pandas later.
# This explicitly tells pandas we are creating a new, independent DataFrame.
pre_code_movies_df = titles_df[is_movie & is_pre_code_era].copy()


# --- 4. Verify the Filtered Data ---
print(f"Found {pre_code_movies_df.shape[0]:,} movies from the Pre-Code era.")
print("\nDataFrame Info:")
pre_code_movies_df.info()

print("\nFirst 5 rows of the filtered DataFrame:")
# In a Jupyter Notebook, display() provides a nicer HTML table format than print().
display(pre_code_movies_df.head())

Pandas version: 2.1.4
Loading data from: ../data/raw_imdb/title.basics.tsv
Successfully loaded title.basics.tsv.
------------------------------
Filtering for Pre-Code era movies (1929-1934)...
Found 12,442 movies from the Pre-Code era.

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 12442 entries, 14911 to 11825202
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          12442 non-null  object 
 1   titleType       12442 non-null  object 
 2   primaryTitle    12442 non-null  object 
 3   originalTitle   12442 non-null  object 
 4   isAdult         12442 non-null  float64
 5   startYear       12442 non-null  float64
 6   endYear         0 non-null      float64
 7   runtimeMinutes  6394 non-null   object 
 8   genres          9677 non-null   object 
dtypes: float64(3), object(6)
memory usage: 972.0+ KB

First 5 rows of the filtered DataFrame:


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
14911,tt0015152,movie,Mustalaishurmaaja,Mustalaishurmaaja,0.0,1929.0,NaN,69,"Drama,Romance"
15787,tt0016035,movie,El lobo,El lobo,0.0,1929.0,NaN,NaN,NaN
17315,tt0017578,movie,The Wrecker,The Wrecker,0.0,1929.0,NaN,74,"Crime,Drama"
17589,tt0017853,movie,Las estrellas,Las estrellas,0.0,1930.0,NaN,NaN,NaN
17801,tt0018069,movie,La del Soto del Parral,La del Soto del Parral,0.0,1929.0,NaN,NaN,NaN


## Part 2: Loading and Merging Personnel Data

Now that we have a clean list of Pre-Code movies, our goal is to identify the **actors, actresses, directors, and writers** associated with them. To do this, we need to load the other IMDb datasets and merge them with our `pre_code_movies_df`.

**Objective:**
Create a single, comprehensive DataFrame that links each Pre-Code movie (`tconst`) to the principal cast/crew members (`nconst`) and their names (`primaryName`).

**Methodology: An Efficient Approach 🧠**
The personnel files (`title.principals.tsv`, `title.crew.tsv`) are enormous, containing data for millions of titles. Loading and then merging these entire files would be very slow and memory-intensive.

A more professional and efficient strategy is to **pre-filter** the personnel data *before* the merge:
1.  **Extract Keys:** Get the unique `tconst` identifiers from our already-filtered `pre_code_movies_df`.
2.  **Filter Personnel Data:** Use these keys to select only the rows from `title.principals` and `title.crew` that match our Pre-Code movies. This drastically reduces the amount of data we need to process.
3.  **Load Name Data:** Load the `name.basics` file, which maps the `nconst` identifiers to actual names.
4.  **Merge:** Perform a series of `left` joins to combine the datasets, ensuring we keep all our Pre-Code movies as the foundation.
5.  **Clean-Up:** Perform minor data type conversions for clarity (e.g., converting `startYear` from a float to an integer).

In [ ]:
# --- 1. Define Paths for Additional Files ---
PRINCIPALS_FILE_PATH = os.path.join(DATA_DIR, 'title.principals.tsv')
NAMES_FILE_PATH = os.path.join(DATA_DIR, 'name.basics.tsv')
CREW_FILE_PATH = os.path.join(DATA_DIR, 'title.crew.tsv')

# --- 2. Load the Additional Datasets ---
print("Loading personnel datasets...")
principals_df = pd.read_csv(PRINCIPALS_FILE_PATH, sep='\t', na_values='\\N')
names_df = pd.read_csv(NAMES_FILE_PATH, sep='\t', na_values='\\N')
# crew_df = pd.read_csv(CREW_FILE_PATH, sep='\t', na_values='\\N') # We can load this later if we focus on writers
print("Personnel datasets loaded.")
print("-" * 30)


# --- 3. Pre-filter `principals_df` for Efficiency ---
# Get the set of unique tconsts from our filtered movie list. Using a set is very fast for lookups.
pre_code_tconsts = set(pre_code_movies_df['tconst'])

print(f"Original size of principals DataFrame: {len(principals_df):,}")

# Keep only the rows in principals_df that correspond to our Pre-Code movies.
pre_code_principals_df = principals_df[principals_df['tconst'].isin(pre_code_tconsts)].copy()

print(f"Filtered size of principals DataFrame: {len(pre_code_principals_df):,}")
print("-" * 30)


# --- 4. Merge the DataFrames ---
print("Merging DataFrames...")

# Step A: Merge our movies list with the filtered principals list.
# A 'left' join ensures we keep all movies from our original pre_code_movies_df.
merged_df = pd.merge(pre_code_movies_df, pre_code_principals_df, on='tconst', how='left')

# Step B: Merge the result with the names list to get the person's name.
# This links the 'nconst' from the principals data to the 'primaryName' in the names data.
final_df = pd.merge(merged_df, names_df, on='nconst', how='left')
print("Merging complete.")
print("-" * 30)


# --- 5. Final Cleaning and Verification ---
# Convert float year to integer for cleaner display. We can do this now as the NaNs are gone.
final_df['startYear'] = final_df['startYear'].astype(int)

# Drop columns we don't need for this analysis to save memory and improve clarity.
columns_to_drop = ['endYear', 'titleType', 'isAdult', 'job', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles']
final_df = final_df.drop(columns=columns_to_drop)


print("Final DataFrame Info:")
final_df.info()

print("\nFirst 10 rows of the final combined DataFrame:")
display(final_df.head(10))

Loading personnel datasets...
